# Phần 2 - Thu thập dữ liệu

## Câu 1

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
import time
import pandas as pd

In [11]:
driver = webdriver.Chrome()

In [12]:
driver.get('https://babycuatoi.vn/xep-hinh-thong-minh')

## Câu 2

In [13]:
while True:
    try:
        nut_xem_them = driver.find_element(by=By.CSS_SELECTOR, value='a#viewmore')
        if nut_xem_them.get_attribute('class') == "viewmore disabled":
            break
        nut_xem_them.click()
        time.sleep(3)
    except:
        break

In [14]:
products = driver.find_elements(by=By.CLASS_NAME, value='product')

In [15]:
import re
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)
def format_price(data):
    return int(data.replace('.','').replace('đ','').replace('₫','').strip())
def format_discount(data):
    return int(data.replace('.','').replace('%','').strip())

In [16]:
lst_names = []
lst_prices = []
lst_discounts = []
lst_descriptions = []
lst_images_names = []
lst_images_urls = []

for product in products:
    name = product.find_element(by=By.CSS_SELECTOR, 
                                value='div.info > a > h3')
    price = product.find_element(by=By.CSS_SELECTOR, 
                                value='div.box-price > p.price')
    discount = product.find_elements(by=By.CSS_SELECTOR, 
                                value='div.box-price > p.label')
    description = product.find_element(by=By.CSS_SELECTOR, 
                                value='div.tooltip-content > p.des')
    image = product.find_element(by=By.CSS_SELECTOR, 
                                value='div.product-content > div.img > a > img')

    lst_names.append(name.text)
    lst_prices.append(format_price(price.text))
    lst_images_urls.append(image.get_attribute('src'))
    lst_images_names.append(image.get_attribute('src').split('/')[-1])
    if len(discount) > 0:
        lst_discounts.append(format_discount(discount[0].text))
    else:
        lst_discounts.append(0)
    lst_descriptions.append(striphtml(description.get_attribute('innerHTML')).strip())

df = pd.DataFrame({
    'Tên sản phẩm': lst_names,
    'Giá bán': lst_prices,
    'Tỷ lệ giảm giá': lst_discounts,
    'Thông tin mô tả': lst_descriptions,
    'Tên tập tin hình sản phẩm': lst_images_names
})

In [131]:
df.head()

,Tên sản phẩm,Giá bán,Tỷ lệ giảm giá,Thông tin mô tả,Tên tập tin hình sản phẩm
0,Đồ chơi xếp hình nam châm thông minh 64 chi ti...,249000,-20,"SẢN PHẨM LÀ HÀNG CAO CẤP, CÓ CHỨNG NHẬN CO,CQ ...",clb-4-do-choi-xep-hinh-nam-cham-64-chi-tiet-ch...
1,Đồ chơi mô hình đoàn tàu kèm bộ số BY-5001,156000,0,- Thương hiệu: BBT GLOBAL\n\n\t\t\t\t\t\t- Kíc...,by-5001-do-choi-doan-tau-vui-nhon-medium.jpg
2,Đồ chơi xếp hình công viên thả bóng vui nhộn 1...,199000,-20,- Thương hiệu: BBT GLOBAL.\n\n\t\t\t\t\t\t- Dà...,6688-123a-do-choi-xep-hinh-thong-minh-cho-be-1...
3,Xếp hình thông minh tàu hỏa tuổi thơ BBT Globa...,263000,-20,- Thương hiệu: BBT Global\n\n\t\t\t\t\t\t- Số ...,5306-do-choi-xep-hinh-cho-be-10-medium.jpg
4,Đồ chơi xếp hình nông trại thả bóng vui nhộn 2...,319000,-20,- Thương hiệu: BBT Global\n\n\t\t\t\t\t\t- Dàn...,6688-123a-do-choi-xep-hinh-thong-minh-cho-be-1...


In [121]:
df.shape

(54, 5)

## Câu 3

In [133]:
path = 'data/Products.csv'
df.to_csv(path, index=False)

## Câu 4

In [127]:
import requests

In [128]:
len(lst_images_names)

54

In [132]:
for index in range(len(lst_images_names)):
    image_name = lst_images_names[index]
    image_url = lst_images_urls[index]
    with open('Images/'+image_name, 'wb') as f:
        res = requests.get(image_url)
        f.write(res.content)

In [9]:
driver.close()